In [1]:
!pip install pymongo[srv] mastodon.py langdetect
import os
import re
import logging
import nltk
import time
import random
from pymongo import MongoClient
from mastodon import Mastodon
from collections import Counter
from langdetect import detect
from datetime import datetime
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
try:
    from google.colab import userdata
except ImportError:
    userdata = None

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('mastodon_data_collection.log')
    ]
)
logger = logging.getLogger(__name__)

# Download NLTK data
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
except Exception as e:
    logger.error(f"Failed to download NLTK data: {str(e)}")
    raise

stop_words = set(stopwords.words('english'))

# MongoDB connection
try:
    username = userdata.get('mongodb_username') if userdata else os.getenv('MONGODB_USERNAME')
    password = userdata.get('mongodb_pw') if userdata else os.getenv('MONGODB_PW')
    if not username or not password:
        raise ValueError("MongoDB credentials not provided")
    cluster_url = "cluster0.8ad48r1.mongodb.net"
    uri = f"mongodb+srv://{username}:{password}@{cluster_url}/?retryWrites=true&w=majority&appName=Cluster0"
    client = MongoClient(uri)
    db = client['social_media_analytics_new']
    tags_collection = db['mastodon_tags_data']
    unique_tag_collection = db['mastodon_unique_tag']
    sentiment_collection = db['mastodon_sentiment_data']
    logger.info("Connected to MongoDB")
except Exception as e:
    logger.error(f"Failed to connect to MongoDB: {str(e)}")
    raise

# Initialize Mastodon API
try:
    mastodon = Mastodon(
        access_token=userdata.get('mastodon_access_token') if userdata else os.getenv('MASTODON_ACCESS_TOKEN'),
        api_base_url="https://mastodon.social"
    )
    logger.info("Connected to Mastodon API")
except Exception as e:
    logger.error(f"Failed to initialize Mastodon API: {str(e)}")
    raise

def preprocess_text(text):
    """Clean and preprocess text for sentiment analysis."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(filtered)

def is_english(text):
    """Check if text is in English."""
    try:
        return detect(text) == 'en'
    except:
        return False

def extract_hashtags(text):
    """Extract hashtags from text."""
    return re.findall(r"#(\w+)", text)

def clear_collections():
    """Delete all documents in mastodon_tags_data, mastodon_unique_tag, and mastodon_sentiment_data."""
    try:
        tags_result = tags_collection.delete_many({})
        unique_tag_result = unique_tag_collection.delete_many({})
        sentiment_result = sentiment_collection.delete_many({})
        logger.info(f"Deleted {tags_result.deleted_count} documents from mastodon_tags_data")
        logger.info(f"Deleted {unique_tag_result.deleted_count} documents from mastodon_unique_tag")
        logger.info(f"Deleted {sentiment_result.deleted_count} documents from mastodon_sentiment_data")
    except Exception as e:
        logger.error(f"Error clearing collections: {str(e)}")
        raise

def fetch_mastodon_trending_tags(top_n_tags=10, fallback_post_limit=1000):
    """Fetch trending tags and store in MongoDB."""
    logger.info("Fetching Mastodon trending tags...")
    try:
        tag_counter = Counter()
        seen_tags = set()

        # Step 1: Fetch trending tags
        trends = mastodon.trending_tags()
        logger.info(f"Found {len(trends)} trending tags from Mastodon")

        for trend in trends:
            raw_tag = trend["name"]
            if not is_english(raw_tag):
                continue
            cleaned_tag = preprocess_text(raw_tag)
            if not cleaned_tag or cleaned_tag in seen_tags:
                continue
            score = sum(int(day.get("uses", 0)) for day in trend.get('history', []))
            tag_counter[cleaned_tag] += score
            seen_tags.add(cleaned_tag)
            tags_collection.insert_one({
                "tag_raw": raw_tag,
                "tag_clean": cleaned_tag,
                "url": trend["url"],
                "score": score,
                "fetched_at": datetime.utcnow()
            })

        # Step 2: Supplement with hashtags from public posts
        if len(tag_counter) < top_n_tags:
            logger.info("Supplementing with hashtags from public posts...")
            public_posts = mastodon.timeline_public(limit=fallback_post_limit)
            for toot in public_posts:
                raw_tags = extract_hashtags(toot["content"])
                for raw_tag in raw_tags:
                    cleaned_tag = preprocess_text(raw_tag)
                    if cleaned_tag and is_english(cleaned_tag) and cleaned_tag not in seen_tags:
                        tag_counter[cleaned_tag] += 1
                        seen_tags.add(cleaned_tag)
                        if len(tag_counter) >= top_n_tags:
                            break
                if len(tag_counter) >= top_n_tags:
                    break

        # Step 3: Final refill if still under top_n_tags
        if len(tag_counter) < top_n_tags:
            logger.info(f"Still under {top_n_tags} — attempting refill...")
            more_posts = mastodon.timeline_public(limit=300)
            for toot in more_posts:
                raw_tags = extract_hashtags(toot["content"])
                for raw_tag in raw_tags:
                    cleaned_tag = preprocess_text(raw_tag)
                    if cleaned_tag and is_english(cleaned_tag) and cleaned_tag not in seen_tags:
                        tag_counter[cleaned_tag] += 1
                        seen_tags.add(cleaned_tag)
                        if len(tag_counter) >= top_n_tags:
                            break
                if len(tag_counter) >= top_n_tags:
                    break

        # Step 4: Insert top N into MongoDB
        final_tags = tag_counter.most_common(top_n_tags)
        logger.info(f"Final unique tag count: {len(final_tags)}")
        for tag, score in final_tags:
            unique_tag_collection.insert_one({
                "tag": tag,
                "score": score,
                "fetched_at": datetime.utcnow()
            })
        logger.info(f"Inserted {len(final_tags)} unique trending tags into 'mastodon_unique_tag'")

        return [tag for tag, _ in final_tags]
    except Exception as e:
        logger.error(f"Error fetching trending tags: {str(e)}")
        return []

def safe_status_context(post_id, retries=3, delay=2):
    """Fetch post context with retries for 503 errors."""
    for attempt in range(retries):
        try:
            return mastodon.status_context(post_id)
        except Exception as e:
            logger.warning(f"Attempt {attempt+1} failed for post {post_id}: {str(e)}. Retrying in {delay} seconds...")
            time.sleep(delay + random.uniform(0, 1))
    logger.error(f"Failed to fetch context for post {post_id} after {retries} retries")
    return {"descendants": []}

def collect_mastodon_data():
    """Collect Mastodon posts/comments and store in MongoDB."""
    try:
        # Clear existing data in collections
        clear_collections()

        # Fetch trending tags
        tags = fetch_mastodon_trending_tags(top_n_tags=10)
        if not tags:
            logger.warning("No trending tags found")
            return
        logger.info(f"Trending Tags: {tags}")

        # Fetch posts and comments for each tag
        for tag in tags:
            logger.info(f"Searching posts for #{tag}")
            posts = mastodon.timeline_hashtag(tag, limit=10)
            for post in posts:
                post_id = post['id']
                post_url = post['url']
                # Fetch replies
                context = safe_status_context(post_id)
                comments = context['descendants']
                for comment in comments:
                    comment_text = comment['content']
                    created_at = comment['created_at']
                    # Strip HTML
                    plain_text = BeautifulSoup(comment_text, "html.parser").get_text()
                    # Preprocess text
                    cleaned_text = preprocess_text(plain_text)
                    if not cleaned_text.strip():
                        logger.warning(f"Skipping comment for post {post_url}: Empty after cleaning")
                        continue
                    # Save to DB
                    doc = {
                        "tag": f"#{tag}",
                        "text": cleaned_text,
                        "created_at": created_at,
                        "post_url": post_url
                    }
                    sentiment_collection.insert_one(doc)
                    logger.info(f"Inserted Mastodon comment for post: {post_url}")
    except Exception as e:
        logger.error(f"Error collecting Mastodon data: {str(e)}")

if __name__ == "__main__":
    logger.info("Starting Mastodon data collection")
    collect_mastodon_data()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ece2ee4a75c4ea9c5242c2a185505002c979952dcfc8ec3d1090121641f9de30
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
